In [63]:
import numpy as np
import pandas as pd
import datetime
import time
import json

In [36]:
data_dir = '../data/spatial/static/'
begin_date, end_date = '2015-01-01', '2019-12-18'
begin_year, end_year = 2015, 2020
regions = ['怀柔区', '大兴区', '昌平区', '通州区', '石景山区', '延庆区', '顺义区', '西城区', '平谷区', '宣武区', '东城区', '丰台区', '朝阳区', '门头沟', '崇文区', '房山区', '密云区', '海淀区']

In [21]:
# festival feature
data = pd.read_csv(data_dir + 'dayfea.txt')
data['date'] = data['date'].apply(lambda x: x.split()[0])
data = data[(data['date'] >= begin_date) & (data['date'] <= end_date)]
data = data.drop(['date'], axis=1)
data = np.array(data)
np.save(data_dir + 'festival_feature.npy', data)

In [70]:
# POIs 

data_old = pd.read_csv(data_dir + 'POI初始版.csv', sep = '\t')
data_old = data_old.drop(['所属城市'], axis=1)
poi_list = data_old.groupby('百度一级行业分类').nunique()
poi_list = list(set(poi_list['百度一级行业分类'].index.tolist()))
old = {}
for region in regions:
    old_temp = []
    for poi in poi_list:
        entry = data_old[(data_old['所属区县'] == region) & (data_old['百度一级行业分类'] == poi)]['数量']
        if len(entry) == 0:
            old_temp.append(0)
        else:
            old_temp.append(int(entry))
    old[region] = old_temp
with open(data_dir + 'old_poi.json',"w") as f:
    json.dump(old,f)
    
data_new = pd.read_csv(data_dir + 'POI初始版.csv', sep = '\t')
data_new = data_new.drop(['所属城市'], axis=1)
new = {}
for region in regions:
    new_temp = []
    for poi in poi_list:
        entry = data_new[(data_old['所属区县'] == region) & (data_new['百度一级行业分类'] == poi)]['数量']
        if len(entry) == 0:
            new_temp.append(0)
        else:
            new_temp.append(int(entry))
    new[region] = new_temp
with open(data_dir + 'new_poi.json',"w") as f:
    json.dump(new,f)

In [114]:
# User portrait

data = pd.read_csv(data_dir + '用户画像_年.csv', sep = '\t', error_bad_lines=False)
data = data.drop(['区', '省'], axis=1).groupby(['日期', '市']).sum().reset_index()
print(len(data))
data_dict = {}
for i in range(len(data)):
    year = data.iloc[i]['日期']
    region = data.iloc[i]['市']
    data_dict[str(year) + region] = list(data.iloc[i][2:].apply(lambda x: x.astype(np.int32)))
with open(data_dir + 'user_portrait.json',"w") as f:
    json.dump(data_dict,f)

90
